<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/langchain-course/07_RAG_implementation_Chain_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Implementation with Vertex AI Gemini Flash and LangChain

## 1. Setup & Installation

In [ ]:
!pip install -q -U langchain-core langchain-community langgraph langchain-chroma requests langchain-openai
!pip install -q -U chromadb langchain-experimental pypdf --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## 2. RAG Part 1 : Extract Transform and Load to Vector DB

In [ ]:
doc_paths = ["https://av.sc.com/corp-en/nr/content/docs/Booklet_IN_2023_v1.pdf",
             "https://av.sc.com/in/content/docs/in-schedule-services-charges.pdf",
             "https://av.sc.com/in/content/docs/in-sc-annual-report.pdf"]

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loaders = [PyPDFLoader(pdf, extract_images=False) for pdf in doc_paths]

docs = []

for loader in loaders:
    doc = loader.load()
    docs.extend(doc)

In [ ]:
len(docs)

46

In [ ]:
docs[1].page_content

'3\n3\n3\n©2021 Morningstar, Inc. All rights reserved. The information in this document is the property of Morningstar, Inc. Reproduction or transcription by any means, in whole or in part, without the prior written \nconsent of Morningstar, Inc., is prohibited.\n The Morningstar RatingTM for Funds    August 2021 Page 2 of 21\ncaptured by standard deviation, as would be the case if excess return were normally or lognormally \ndistributed, which is not always the case. Also, standard deviation measures variation both above \nand below the mean equally. But investors are generally risk-averse and dislike downside variation \nmore than upside variation. Morningstar gives more weight to downside variation when calculating \nMorningstar Risk-Adjusted Return and does not make any assumptions about the distribution of \nexcess returns.\nThe other commonly accepted meaning of “risk-adjusted” is based on assumed investor preferences. \nUnder this approach, higher return is “good” and higher ris

In [ ]:
# drop pages which have less than 100 characters (e.g. header pages, empty separater pages)
docs = [doc for doc in docs if len(doc.page_content.strip())>100]
len(docs)

45

In [ ]:
# For all remaining document, checking the average characters count
sum(len(doc.page_content) for doc in docs)/len(docs)

2657.3333333333335

In [ ]:
# split the documents(each pdf page as one document) , into multiple so that at max there are 3500 characters in one document, with overlap of 500 characters while splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=500)
docs = text_splitter.split_documents(docs)
len(docs)

53

In [ ]:
print(docs[1].page_content)

3
3
3
©2021 Morningstar, Inc. All rights reserved. The information in this document is the property of Morningstar, Inc. Reproduction or transcription by any means, in whole or in part, without the prior written 
consent of Morningstar, Inc., is prohibited.
 The Morningstar RatingTM for Funds    August 2021 Page 2 of 21
captured by standard deviation, as would be the case if excess return were normally or lognormally 
distributed, which is not always the case. Also, standard deviation measures variation both above 
and below the mean equally. But investors are generally risk-averse and dislike downside variation 
more than upside variation. Morningstar gives more weight to downside variation when calculating 
Morningstar Risk-Adjusted Return and does not make any assumptions about the distribution of 
excess returns.
The other commonly accepted meaning of “risk-adjusted” is based on assumed investor preferences. 
Under this approach, higher return is “good” and higher risk is “bad” und

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# loading embedding model from Hugging Face
from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model_name = "BAAI/bge-large-en-v1.5"
#embeddings = HuggingFaceBgeEmbeddings(model_name=embedding_model_name,)

In [ ]:
# initializng the vector store using CHromaDB
from langchain_chroma import Chroma
# Using embedding model, to embed documents to vector and store to a vector db
vectorstore = Chroma.from_documents(documents = docs[:20], embedding=embedding_fun)

# using vector db object to initialize a retriever object - to perform vector search/retrieval
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [ ]:
len(vectorstore.get()['ids'])

20

In [ ]:
# adding more docs to the vector database
vectorstore.add_documents(documents=docs[20:40],embedding=embedding_fun)

['7c519a1a-9ff2-4a9b-8378-21ad25890a6f',
 'c241de41-339a-439c-b336-74b6e1fe4dc6',
 'e47fa05c-6089-4e1d-ae96-667f34b5aefe',
 '9bfcb2dc-31d8-41de-9086-b15366707c4f',
 '79ba9f2b-c94b-4fa4-91cc-96722c075b7b',
 'c1fb30ea-7dcb-424d-84ab-a2f12ceaeac7',
 '1f359dca-6446-4efb-8913-b8ef1ebe2601',
 'f6a21875-1f32-490c-be29-8374be4271c4',
 'f2c490db-1ef5-45a7-ad41-ecc206d77664',
 '7a25f20b-3fa6-4b77-9634-14b6f705abbf',
 '47c10578-db1c-43d1-9d41-6f44a210bb47',
 'b954ef28-4c49-41b7-8b4c-93b76b778c4b',
 'fc3ca672-5c57-49a3-be85-2384f88a81b5',
 'a937804d-1ccb-4e84-b5d0-630c18258649',
 '55a40a90-7e0e-4a18-ad3c-e4b292c4ba97',
 '37116c1e-026c-415a-ad56-2e60d78eeea3',
 '7a9558e4-c46a-4939-aff1-c2d15916b6d4',
 'd7bf4bc2-fd1f-4145-bc1f-3a3f365ed86c',
 '94dc315c-1dbc-4142-8e70-e355bcc92665',
 'ef7ee071-2acb-4adc-8243-ef4512798b4b']

In [ ]:
# adding more docs to the vector database
vectorstore.add_documents(documents=docs[40:],embedding=embedding_fun)

['1d8a26c3-93de-424a-b812-93a64688fa61',
 '874792b1-b1b1-4af6-8e06-f79accd69a77',
 'e806d912-4fb5-4474-9973-db60a804b6c7',
 'ca7fbdfd-b66c-4141-a3ad-237e1283fb1b',
 'c8f346e5-a85b-4e3b-bcff-c692bb4fad16',
 '08f02cb7-af54-4d8b-825f-0969450daf83',
 '5ee9057f-ce03-49cc-820a-c4dd035270ba',
 'ddb41781-185e-49bd-858c-ce0d39a8466c',
 'a417b975-1c81-4205-af06-4068826940d5',
 'f838f208-2cdc-4692-9f90-19e3002aa249',
 'd7e414fb-c646-450c-9580-a09576618d41',
 'ed8fd261-fdc2-438c-9454-dba79873a189',
 'cef2b288-8274-4640-a2c4-48e13dac57fc']

In [ ]:
len(vectorstore.get()['ids'])

53

In [ ]:
retrieved_docs = retriever.invoke("What is Large Cap equity fund")
len(retrieved_docs)

2

In [ ]:
print(retrieved_docs[0].page_content)

?  
 
 
 
 
 
 
 
 
 
 
 Categor y Definitions  
India 
Equity  
 
Large-Cap 
Large-Cap funds primarily consist of stocks which a re the Top 100 stocks by full market capitalization  of 
the equity market. These funds invest at least 80% of total assets in Indian equities and the balance can 
be invested in other asset classes such as fixed in come and overseas equities, among others. Funds in 
this category would invest at least 80% of their to tal assets in large-cap stocks. 
Morningstar Category Index: S&P BSE 100 TR 
 
Mid-Cap 
Mid-Cap funds primarily consist of stocks ranked 10 1st to 250th by full market capitalization of the 
equity market. These funds invest at least 65% of t otal assets in Indian equities, and the balance can  be 
invested in other asset classes such as fixed incom e and overseas equities, among others. Funds in thi s 
category would invest at least 65% of their total a ssets in mid-cap stocks. 
Morningstar Category Index: S&P BSE Mid Cap TR 
 
Small-Cap 
Smal

In [ ]:
print(retrieved_docs[1].page_content)

©2019 Morningstar, Inc. All rights reserved. The in formation in this document is the property of Morni ngstar, Inc. Reproduction or transcription by any m eans, in whole or part, without  
the prior written consent of Morningstar, Inc., is prohibited. Category Definitions , India  | 26 February 2021  Page 2 of 12  Multi- Cap Multi-Cap funds invest at least 75% of their total assets in Indian equities, and the balance can be invested in other asset classes such as fixed incom e and overseas equities, among others. These funds will invest a minimum of 25% each in Large Cap, Mid  Cap and Small Cap stocks. Morningstar Category Index: S&P BSE 500 TR   Large & Mid- Cap Large & Mid-Cap funds primarily consist of stocks w hich are the Top 250 stocks by full market capitalization of the equity market. These funds in vest at least 70% of total assets in Indian equitie s and the balance can be invested in other asset classes such as fixed income and overseas equities, among others. Funds in this

### Implementing RAG Chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

In [ ]:
from langchain_openai import ChatOpenAI

chatmodel = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0.5)

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | chatmodel


In [ ]:
response = rag_chain.invoke("tell me about mid cap market")

print(response.content)

Mid-Cap funds primarily invest in stocks ranked 101st to 250th by full market capitalization. They must invest at least 65% of their total assets in Indian equities, with the remaining portion potentially invested in other asset classes like fixed income and overseas equities.  The Morningstar Category Index for Mid-Cap funds is the S&P BSE Mid Cap TR. 



### Implementing RAG Agent

### Creating retriever Tool

In [ ]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "searchCapitalMarket",
    "Searches and returns excerpts about trading stocks markets shares capital markets, DO not use it for any other info than that of capital market/ finance questions",
)

In [ ]:
from langchain_openai import ChatOpenAI

chatmodel = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0.5)

In [ ]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(chatmodel, [tool,])

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
query = "Hi"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello! 👋 How can I help you today? 😊 \n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 46, 'candidates_token_count': 13, 'total_token_count': 59}}, id='run-fdbea2eb-b69f-4cf3-bdd9-af153b681518-0', usage_metadata={'input_tokens': 46, 'output_tokens': 13, 'total_tokens': 59})]}}
----


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
query = "What is mid cap capital market?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'searchCapitalMarket', 'arguments': '{"query": "What is mid cap capital market?"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 52, 'candidates_token_count': 11, 'total_token_count': 63}}, id='run-3c7f731a-6e44-4dcd-88cf-63459926489f-0', tool_calls=[{'name': 'searchCapitalMarket', 'args': {'query': 'W

{'tools': {'messages': [ToolMessage(content='?  \n \n \n \n \n \n \n \n \n \n \n Categor y Definitions  \nIndia \nEquity  \n \nLarge-Cap \nLarge-Cap funds primarily consist of stocks which a re the Top 100 stocks by full market capitalization  of \nthe equity market. These funds invest at least 80% of total assets in Indian equities and the balance can \nbe invested in other asset classes such as fixed in come and overseas equities, among others. Funds in \nthis category would invest at least 80% of their to tal assets in large-cap stocks. \nMorningstar Category Index: S&P BSE 100 TR \n \nMid-Cap \nMid-Cap funds primarily consist of stocks ranked 10 1st to 250th by full market capitalization of the \nequity market. These funds invest at least 65% of t otal assets in Indian equities, and the balance can  be \ninvested in other asset classes such as fixed incom e and overseas equities, among others. Funds in thi s \ncategory would invest at least 65% of their total a ssets in mid-cap sto

{'agent': {'messages': [AIMessage(content='A mid-cap capital market refers to a segment of the stock market that includes companies with a market capitalization between $2 billion and $10 billion. These companies are considered to be larger than small-cap companies but smaller than large-cap companies. \n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 797, 'candidates_token_count': 53, 

## Multi-tool RAG Agent

In [ ]:
!pip install wikipedia --quiet

  Preparing metadata (setup.py) ... done


In [ ]:
from langchain_openai import ChatOpenAI

chatmodel = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0.5)

In [ ]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

ragtool = create_retriever_tool(
    retriever,
    "searchCapitalMarket",
    "Searches and returns excerpts about trading stocks markets shares capital markets, DO not use it for any other info than that of capital market/ finance questions",
)


tools = [ragtool,wiki]

In [ ]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(chatmodel, tools)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
query = "Hi"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello! 👋 How can I help you today? 😊 \n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 99, 'candidates_token_count': 13, 'total_token_count': 112}}, id='run-7ff5f37b-66b1-436f-b24a-d90ee09ddac3-0', usage_metadata={'input_tokens': 99, 'output_tokens': 13, 'total_tokens': 112})]}}
----


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
query = "What is mid cap capital market?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'searchCapitalMarket', 'arguments': '{"query": "What is mid cap capital market?"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 105, 'candidates_token_count': 11, 'total_token_count': 116}}, id='run-b01d40f0-a1a8-4381-ba5c-4e9fad080d55-0', tool_calls=[{'name': 'searchCapitalMarket', 'args': {'query': 

{'tools': {'messages': [ToolMessage(content='?  \n \n \n \n \n \n \n \n \n \n \n Categor y Definitions  \nIndia \nEquity  \n \nLarge-Cap \nLarge-Cap funds primarily consist of stocks which a re the Top 100 stocks by full market capitalization  of \nthe equity market. These funds invest at least 80% of total assets in Indian equities and the balance can \nbe invested in other asset classes such as fixed in come and overseas equities, among others. Funds in \nthis category would invest at least 80% of their to tal assets in large-cap stocks. \nMorningstar Category Index: S&P BSE 100 TR \n \nMid-Cap \nMid-Cap funds primarily consist of stocks ranked 10 1st to 250th by full market capitalization of the \nequity market. These funds invest at least 65% of t otal assets in Indian equities, and the balance can  be \ninvested in other asset classes such as fixed incom e and overseas equities, among others. Funds in thi s \ncategory would invest at least 65% of their total a ssets in mid-cap sto

{'agent': {'messages': [AIMessage(content='Mid-cap funds primarily consist of stocks ranked 101st to 250th by full market capitalization of the equity market. These funds invest at least 65% of total assets in Indian equities, and the balance can be invested in other asset classes such as fixed income and overseas equities, among others. Funds in this category would invest at least 65% of their total assets in mid-cap stocks. \n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked':

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
query = "What is the capital city of Philippines?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'wikipedia', 'arguments': '{"query": "Philippines"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 106, 'candidates_token_count': 3, 'total_token_count': 109}}, id='run-0d6ec0fb-3f4e-4ab0-aa01-538d2dfab7da-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Philippines'}, 'id': 'f972bbe7-353f-4720

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


{'tools': {'messages': [ToolMessage(content="Page: Philippines\nSummary: The Philippines, officially the Republic of the Philippines, is an archipelagic country in Southeast Asia. In the western Pacific Ocean, it consists of 7,641 islands, with a total area of 300,000 square kilometers, which are broadly categorized in three main geographical divisions from north to south: Luzon, Visayas, and Mindanao. The Philippines is bounded by the South China Sea to the west, the Philippine Sea to the east, and the Celebes Sea to the south. It shares maritime borders with Taiwan to the north, Japan to the northeast, Palau to the east and southeast, Indonesia to the south, Malaysia to the southwest, Vietnam to the west, and China to the northwest. It is the world's twelfth-most-populous country, with diverse ethnicities and cultures. Manila is the country's capital, and its most populated city is Quezon City. Both are within Metro Manila.\nNegritos, the archipelago's earliest inhabitants, were foll